In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Set up the path to your dataset
dataset_dir = '/kaggle/input/alzheimers-disease-classification/Alzheimer\'s Disease/'

# Function to load images and labels from subfolders
def load_images_and_labels(dataset_dir):
    images = []
    labels = []
    for label, subfolder in enumerate(os.listdir(dataset_dir)):
        subfolder_path = os.path.join(dataset_dir, subfolder)
        if os.path.isdir(subfolder_path):
            for img_name in os.listdir(subfolder_path):
                if img_name.endswith('.jpg') or img_name.endswith('.png'):
                    img_path = os.path.join(subfolder_path, img_name)
                    img = image.load_img(img_path, target_size=(224, 224))
                    img_array = image.img_to_array(img) / 255.0
                    images.append(img_array)
                    labels.append(label)
    return np.array(images), np.array(labels)

# Load all images and labels
images, labels = load_images_and_labels(dataset_dir)

# Split the data into training (80%), validation (10%), and testing (10%)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Define MobileNetV2 Transfer Learning Model
def build_mobilenetv2_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
mobilenet_model = build_mobilenetv2_model()
mobilenet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, verbose=1)

# Fine-tuning (Fusion Shot Algorithm)
def fusion_shot_algorithm(model, X_train, y_train, X_val, y_val):
    for layer in model.layers[-10:]:
        layer.trainable = True
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=65, batch_size=32, verbose=1)
    return model

mobilenet_model = fusion_shot_algorithm(mobilenet_model, X_train, y_train, X_val, y_val)

# Evaluate on test set
y_pred = np.argmax(mobilenet_model.predict(X_test), axis=1)
print("Final Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
y_prob = mobilenet_model.predict(X_test)
y_true_binary = tf.keras.utils.to_categorical(y_test, num_classes=2)
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_true_binary[:, 1], y_prob[:, 1])
plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_true_binary[:, 1], y_prob[:, 1]):.4f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# Precision, Recall, F1 Score
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Test Accuracy
loss, acc = mobilenet_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc:.4f}')
